In [ ]:
!pip install openai python-dotenv pypdf gradio

In [21]:
from dotenv import load_dotenv
from IPython.display import Markdown
from openai import OpenAI
from pypdf import PdfReader
import os
import pandas as pd
import gradio as gr

In [23]:
load_dotenv(override=True)

True

In [24]:
openaiapi = os.getenv("OPENAI_API_KEY")
groqapi = os.getenv("groq_api_key")

In [25]:
if openaiapi:
    print(f"Openai key is found and starts with: {openaiapi[:8]}")
else:
    print("Open ai api not found")

if groqapi:
    print(f"groqapi key is found and starts with: {groqapi[:8]}")
else:
    print("groqapi ai api not found")

Openai key is found and starts with: sk-proj-
groqapi key is found and starts with: gsk_Vopn


In [26]:
# System prompt tailored for property recommendations
system_prompt_property = """
You are a real estate advisor AI. 
You are given a dataset of properties including location, price, bedrooms, bathrooms, parking spaces, toilets, title, and description. 
When a user provides their preferences (budget, location, property type, number of bedrooms), you must return the top 3 properties that best match their needs. 
Provide a short reasoning for each recommendation. 
Always consider price affordability and feature match.
"""


In [27]:
import pandas as pd

sheet_url = "https://docs.google.com/spreadsheets/d/15h3j-Q-Xepsok2ru5Au_havdWMdJ983qBGBXzceCuig/export?format=csv&gid=474984245"
df_cleaned = pd.read_csv(sheet_url)

print(df_cleaned.head())


                                            Location Currency         Price  \
0  Urban Prime Three Phase 2 Estate, Ogombo, Ajah...        ₦  1.500000e+08   
1                                       Ikoyi, Lagos        ₦  4.290552e+09   
2          Pinnock Beach Estate, Osapa, Lekki, Lagos        ₦  1.400000e+09   
3        Oseni Mayegun Street, Ologolo, Lekki, Lagos        ₦  2.000000e+08   
4                   Ikate Lekki, Ikate, Lekki, Lagos        ₦  2.100000e+08   

      Contact  Bedrooms                                      Realtor  \
0  8067495852         5                            Lake Pad Ventures   
1  7065266133         6                                   Bera Deals   
2  7065266133         6                                   Bera Deals   
3  8079792003         4                            Paul Osaji And Co   
4  8084677480         5  Jasmine Real Properties And Estate Services   

   Bathrooms                                           Title  \
0          6          Luxury

In [ ]:
#  existing functions
client = OpenAI()

def get_recommendation_input(budget=None, location=None, property_type=None, bedrooms=None):
    """
    Prepare user query and filtered dataset subset for LLM
    """
    df_filtered = df_cleaned  # assume df_cleaned is globally available

    # Filters
    if budget is not None:
        df_filtered = df_filtered[df_filtered['Price'] <= budget]
    if location:
        df_filtered = df_filtered[df_filtered['Location'].str.contains(location, case=False, na=False)]
    if property_type:
        df_filtered = df_filtered[df_filtered['Title'].str.contains(property_type, case=False, na=False)]
    if bedrooms is not None:
        df_filtered = df_filtered[df_filtered['Bedrooms'] == bedrooms]

    # Select top 5
    df_top = df_filtered.sort_values(by='Score', ascending=False).head(5)

    # Build user prompt
    prompt = f"Recommend top properties based on these preferences:\nBudget: {budget}\n"
    prompt += f"Location: {location}\nProperty Type: {property_type}\nBedrooms: {bedrooms}\n\n"
    prompt += "Available properties:\n"
    for idx, row in df_top.iterrows():
        prompt += f"- {row['Title']} | Price: ₦{row['Price']:,.0f} | Location: {row['Location']} | Bedrooms: {row['Bedrooms']} | Bathrooms: {row['Bathrooms']} | Parking: {row['Parking Spaces']} | Toilets: {row['Toilets']} | Score: {row['Score']}\n"
    return prompt


def chat_property(user_message, history):
    """
    Chat wrapper that keeps history and calls GPT
    """
    # Convert Gradio history format [(user, bot), ...] to OpenAI messages
    messages = [{"role": "system", "content": system_prompt_property}]
    for user, bot in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": bot})
    messages.append({"role": "user", "content": user_message})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    reply = response.choices[0].message.content
    return reply


# Function to link filters + chat
def recommend_with_filters(budget, location, property_type, bedrooms, history=[]):
    user_prompt = get_recommendation_input(budget, location, property_type, bedrooms)
    reply = chat_property(user_prompt, history)
    history.append((user_prompt, reply))
    return history, history


# Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## 🏡 Property Finder Chatbot")

    with gr.Row():
        budget = gr.Number(label="Budget (₦)", value=200000000)
        location = gr.Textbox(label="Location", placeholder="e.g., Lagos")
        property_type = gr.Textbox(label="Property Type", placeholder="e.g., Apartment")
        bedrooms = gr.Number(label="Bedrooms", value=3)

    chatbot = gr.Chatbot()
    state = gr.State([])

    send_button = gr.Button("Find Properties")

    send_button.click(
        recommend_with_filters,
        inputs=[budget, location, property_type, bedrooms, state],
        outputs=[chatbot, state]
    )

demo.launch()


C:\Users\Admin\AppData\Local\Temp\ipykernel_6860\4274408872.py:69: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
